# Topic Modeling

For more details on how topic modeling works, [see here](https://topix.io/tutorial/tutorial.html)

### Execute this cell to install required python module

After you've installed this once, you can delete this cell.

In [1]:
!pip install pyldavis

  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97716 sha256=8f9c2543a8d2d725b5c79445246c2e9eeb26aa2db4da9c34c93ad06c65c59d7e
  Stored in directory: c:\users\reset\appdata\local\pip\cache\wheels\3b\fb\41\e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyldavis


### Import dependencies

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups

# module to visualize topics
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

/Users/yanxu/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/Users/yanxu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/yanxu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any beha

### Load 20newsgroups data

In [2]:
news = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame({"body": news.data})
df.shape

(11314, 1)

### Preprocess text

In [3]:
from utils import clean_text
df['body'] = df['body'].apply(lambda x: clean_text(x))

### Generate feature vectors

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(stop_words='english')
tf = tf_vectorizer.fit_transform(df['body'])
tf_feature_names = tf_vectorizer.get_feature_names()

### Fit feature vectors to the LDA topic model

In [5]:
from sklearn.decomposition import LatentDirichletAllocation

no_topics = 20

lda = LatentDirichletAllocation(n_components=no_topics, random_state=4, evaluate_every=1).fit(tf)

### Display top words for each topic

In [6]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic: {topic_idx}")
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, tf_feature_names, no_top_words)

Topic: 0
year israel think good israeli team game right time people
Topic: 1
think question believe point science true exist atheists make know
Topic: 2
government encryption chip clipper keys used security people right public
Topic: 3
armenian turkish armenians people turkey jews turks genocide armenia government
Topic: 4
game team play hockey games period season players league teams
Topic: 5
file information output internet email program anonymous space list data
Topic: 6
file firearms control crime police weapons guns weapon state firearm
Topic: 7
know judas bible fallacy hell argument really doug reply read
Topic: 8
file window available image program files using version windows software
Topic: 9
people jesus think know believe time life good bible christian
Topic: 10
cubs allocation greek cross suck john linked unit baltimore moncton
Topic: 11
april lines university states send santa sweden available group rates
Topic: 12
darren hawks company maria tomb station redesign patients c

### Visualizing our topics in 2-dimensional space

How to interpret this visualization:
1. Each bubble represents a topic
2. Larger topics are more frequent in the corpus
3. Topics closer together are more similar
4. When you click on a topic, the most relevant terms for that topic show in red on the right, and in blue is the frequency of that term in all other topics
5. When you hover over a word in the chart on the right, the bubbles will adjust according to how relevant that term is to each topic


In [7]:
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.183746  0.128524       1        1  13.956314
13     0.165953  0.120940       2        1  13.676325
8      0.150775 -0.187149       3        1  10.504338
15     0.180814 -0.106348       4        1   9.304981
0      0.110238  0.153290       5        1   7.559274
2      0.142393 -0.026679       6        1   6.982614
5      0.070191 -0.158635       7        1   5.499671
19     0.022713 -0.093651       8        1   5.113681
1      0.128134  0.077964       9        1   4.723539
3     -0.011411  0.138392      10        1   3.943775
6     -0.025740  0.000595      11        1   3.925342
4     -0.086941  0.114111      12        1   3.253322
14     0.030211 -0.151920      13        1   2.499967
18    -0.062466  0.000548      14        1   1.996939
7     -0.052311  0.052158      15        1   1.362255
12    -0.175666  0.017912      16        1   1.240547
11    -0.147126 -0.041645      17        1   1.238379
16    -0.240892 -0.030165      18        1   1.103570
17    -0.215109 -0.024638      19        1   1.071123
10    -0.167508  0.016396      20        1   1.044044, topic_info=             Term         Freq        Total Category  logprob  loglift
20095       drive  1053.000000  1053.000000  Default  30.0000  30.0000
25052        file  1564.000000  1564.000000  Default  29.0000  29.0000
70677       space  1018.000000  1018.000000  Default  28.0000  28.0000
27341        game   716.000000   716.000000  Default  27.0000  27.0000
74592        team   688.000000   688.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
41489    limbaugh    11.263197    33.684532  Topic20  -6.6847   3.4666
25877        food    16.949251   261.244885  Topic20  -6.2760   1.8268
79089  university    16.308242   546.338880  Topic20  -6.3146   1.0505
2372    americans    13.571218   165.796268  Topic20  -6.4983   2.0593
22994      ethnic    11.572884    63.654653  Topic20  -6.6576   2.8573

[1405 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
218        1  0.214608      absolute
218        2  0.024762      absolute
218        3  0.033017      absolute
218        5  0.057779      absolute
218        6  0.024762      absolute
...      ...       ...           ...
87749      1  0.037691       zionist
87749      5  0.565363       zionist
87749     16  0.339218       zionist
88024     11  0.063150  zoroastrians
88024     15  0.884099  zoroastrians

[5611 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 14, 9, 16, 1, 3, 6, 20, 2, 4, 7, 5, 15, 19, 8, 13, 12, 17, 18, 11])

### Create Document - Topic Matrix

In [8]:
lda_output = lda.transform(tf)

# column names
topicnames = ["Topic" + str(i) for i in range(no_topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(df))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.290000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.370000,0.000000,0.310000,0.000000,0.000000,0.000000,0.000000,13
Doc1,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.540000,0.000000,0.000000,0.000000,0.000000,15
Doc2,0.030000,0.050000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.320000,0.000000,0.480000,0.000000,0.000000,0.000000,0.000000,15
Doc3,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.840000,0.010000,0.010000,0.010000,0.010000,0.010000,14
Doc4,0.000000,0.000000,0.000000,0.110000,0.000000,0.000000,0.000000,0.000000,0.430000,0.330000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.110000,8
Doc5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.110000,0.000000,0.160000,0.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9
Doc6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.670000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13
Doc7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.310000,0.690000,0.000000,0.000000,0.000000,0.000000,15
Doc8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.210000,0.000000,0.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8
Doc9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.180000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.550000,0.260000,0.000000,0.000000,0.000000,15
